# ARRANEGE COVARIABLES

### 1. Get and arrange data
### 2 Define Hyperthension from first ocurrences
### 3. Define other variables from first instances
### 4. Define DT2
### 5. Define Hypertension
### 6. Define Dyslipidemia
### 7. Define BMI
### 8. Save output

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from io import StringIO
import seaborn as sns
import scipy.stats as stats
pd.set_option('display.max_columns', 150)
pd.set_option('display.max_rows', 40)

import warnings
warnings.filterwarnings('ignore', category=pd.errors.DtypeWarning)
warnings.filterwarnings('ignore', category=pd.errors.PerformanceWarning)

import matplotlib as mpl
mpl.rcParams['figure.dpi']= 300

## 1. Get and arrange data

In [2]:
### OPEN CANCER DATA generated following the steps from the README.md
### Data-fields are indicated in the paper
cvd_ukb200k = pd.read_csv('/workspace/datasets/ukbiobank_ch/ukb_data_670124/clinical_data_ukb670124/cvd_data_670124.txt',
                              sep="\t")
len(cvd_ukb200k)

502394

In [8]:
# All variable codes
colnoms = []
for x in cvd_ukb200k.columns.tolist():
    colnoms.append(x.split('-')[0])
set(colnoms)

{'130708',
 '130709',
 '130712',
 '130713',
 '130714',
 '130715',
 '131270',
 '131271',
 '131272',
 '131273',
 '131274',
 '131275',
 '131276',
 '131277',
 '131278',
 '131279',
 '131280',
 '131281',
 '131282',
 '131283',
 '131284',
 '131285',
 '131286',
 '131287',
 '131288',
 '131289',
 '131290',
 '131291',
 '131292',
 '131293',
 '131294',
 '131295',
 '131296',
 '131297',
 '131298',
 '131299',
 '131300',
 '131301',
 '131302',
 '131303',
 '131304',
 '131305',
 '131306',
 '131307',
 '131308',
 '131309',
 '131310',
 '131311',
 '131312',
 '131313',
 '131314',
 '131315',
 '131316',
 '131317',
 '131318',
 '131319',
 '131320',
 '131321',
 '131322',
 '131323',
 '131324',
 '131325',
 '131326',
 '131327',
 '131328',
 '131329',
 '131330',
 '131331',
 '131332',
 '131333',
 '131334',
 '131335',
 '131336',
 '131337',
 '131338',
 '131339',
 '131340',
 '131341',
 '131342',
 '131343',
 '131344',
 '131345',
 '131346',
 '131347',
 '131348',
 '131349',
 '131350',
 '131351',
 '131352',
 '131353',
 '131354',

In [9]:
# Function to obtain the index from columns
def columna(data, name):
    a = []
    for n, x in enumerate(data.columns.tolist()):
        if str(x).startswith(str(name)):
            a.append([n , x])
    return(a)

In [10]:
columna(cvd_ukb200k, '41270')

[[1213, '41270-0.0'],
 [1214, '41270-0.1'],
 [1215, '41270-0.2'],
 [1216, '41270-0.3'],
 [1217, '41270-0.4'],
 [1218, '41270-0.5'],
 [1219, '41270-0.6'],
 [1220, '41270-0.7'],
 [1221, '41270-0.8'],
 [1222, '41270-0.9'],
 [1223, '41270-0.10'],
 [1224, '41270-0.11'],
 [1225, '41270-0.12'],
 [1226, '41270-0.13'],
 [1227, '41270-0.14'],
 [1228, '41270-0.15'],
 [1229, '41270-0.16'],
 [1230, '41270-0.17'],
 [1231, '41270-0.18'],
 [1232, '41270-0.19'],
 [1233, '41270-0.20'],
 [1234, '41270-0.21'],
 [1235, '41270-0.22'],
 [1236, '41270-0.23'],
 [1237, '41270-0.24'],
 [1238, '41270-0.25'],
 [1239, '41270-0.26'],
 [1240, '41270-0.27'],
 [1241, '41270-0.28'],
 [1242, '41270-0.29'],
 [1243, '41270-0.30'],
 [1244, '41270-0.31'],
 [1245, '41270-0.32'],
 [1246, '41270-0.33'],
 [1247, '41270-0.34'],
 [1248, '41270-0.35'],
 [1249, '41270-0.36'],
 [1250, '41270-0.37'],
 [1251, '41270-0.38'],
 [1252, '41270-0.39'],
 [1253, '41270-0.40'],
 [1254, '41270-0.41'],
 [1255, '41270-0.42'],
 [1256, '41270-0.43']

In [11]:
#remove columns with all values missed

cvd_ukb200k = cvd_ukb200k.dropna(how='all', axis=1)
cvd_ukb200k.reset_index(inplace=True, drop=True)

## 2 Define Hyperthension from first ocurrences

In [12]:
##### 
##### SUMMARY DIAGNOSIS   
#####

# 41202 [666:745]  Diagnoses - main ICD10
# 41262 [828:907]  Date of first in-patient diagnosis - main ICD10
# 41203 [745:773]   Diagnoses - main ICD9
# 41263 [907:935]   Date of first in-patient diagnosis - main ICD9
# 41270 [935:1178]  Diagnoses - ICD10
# 41280 [1349:1592] Date of first in-patient diagnosis - ICD10
# 41271 [1178:1225] Diagnoses - ICD9
# 41281 [1592:1638] Date of first in-patient diagnosis - ICD9

# to remove
# 1900-01-01 represents "Code has no event date"
# 1901-01-01 represents "Code has event date before participant's date of birth"
# 1902-02-02 represents "Code has event date matching participant's date of birth"
# 1903-03-03 represents "Code has event date after participant's date of birth and falls in the same calendar year as date of birth"
# 2037-07-07 represents "Code has event date in the future and is presumed to be a place-holder or other system default"
remove_dates = ['1900-01-01', '1901-01-01', '1902-02-02', '1903-03-03', '2037-07-07']

# define columns
column_diseases = list(range(666,745)) + list(range(745,773)) + list(range(935,1178)) + list(range(1178,1225))
column_dates = list(range(828,907)) + list(range(907,935)) + list(range(1349,1592)) + list(range(1592,1638))

def Diagnose_variable(patient):
       
    # select columns
    diseases = patient[column_diseases]
    dates = patient[column_dates]
    
    Hypertension = [i for i in diseases if str(i).startswith(icd_10_9)]
    i_Hypertension_disease = [i for i,j in enumerate(diseases) if str(j).startswith(icd_10_9)]  
    dates_hypertension_disease = [dates[index] for index in i_Hypertension_disease]
    if len(dates_hypertension_disease) > 0:
        
        for i in range(len(dates_hypertension_disease)):
            if dates_hypertension_disease[i] in remove_dates:
                dates_hypertension_disease[i] = 'nan'
        
        mindate = min(dates_hypertension_disease)
        i_minage = [i for i,j in enumerate(dates_hypertension_disease) if str(j).startswith(str(mindate))]  
        Hypertens = [str(Hypertension[index]) for index in i_minage]
        
        if mindate == 'nan':
            return np.nan
        else:
            return ', '.join(set(Hypertens))
        
    else:
        return np.nan
    
def Diagnose_date(patient):
       
    # select columns
    diseases = patient[column_diseases]
    dates = patient[column_dates]
    
    Hypertension = [i for i in diseases if str(i).startswith(icd_10_9)]
    i_Hypertension_disease = [i for i,j in enumerate(diseases) if str(j).startswith(icd_10_9)]  
    dates_hypertension_disease = [dates[index] for index in i_Hypertension_disease]
    if len(dates_hypertension_disease) > 0:
        
        for i in range(len(dates_hypertension_disease)):
            if dates_hypertension_disease[i] in remove_dates:
                dates_hypertension_disease[i] = 'nan'
        
        mindate = min(dates_hypertension_disease)
        if mindate == 'nan':
            return np.nan
        else:
            return mindate
    else:
        return np.nan

In [ ]:
# Define I70, Atherosclerosis
icd_10_9 = ('440', 'I70')
cvd_ukb200k['I70'] = cvd_ukb200k.apply(lambda x: Diagnose_variable(x), axis=1)
cvd_ukb200k['I70_date'] = cvd_ukb200k.apply(lambda x: Diagnose_date(x), axis=1)

## 3. Define other variables from first instances

In [ ]:
df1 = cvd_ukb200k.copy()

In [2]:
from tqdm import tqdm
variables = [
 '130708-0.0',          
 '130712-0.0',
 '130714-0.0',
 '131286-0.0',
 '131288-0.0',
 '131290-0.0',
 '131292-0.0',
 '131294-0.0',
 'I70_date']
for variable in tqdm(variables):
    years = []
    for n, x in enumerate(df1[variable]):
        if x is np.nan:
            years.append(x)
        elif x in remove_dates:
            years.append(np.nan)
        else:
            year = pd.to_datetime(x, format='%Y-%m-%d')-pd.to_datetime(df1.loc[n,'53-0.0'], format='%Y-%m-%d')
            years.append(year/np.timedelta64(1,'Y'))
    var = variable.split('_')[0]
    var = var.split('-')[0]
    df1[var+'_years'] = years
    df1[var+'_var'] = np.where(df1[var+'_years'].notnull(), 1, 0)
    df1[var+'_post'] = np.where(df1[var+'_years'] > 0, 1, 0)
    df1[var+'_pre'] = np.where(df1[var+'_years'] < 0, 1, 0)

## 4. Define DT2

In [86]:
# Diabetes tipe II from first occurances
# If date of first ocurrence in E11, E13 or E14 > 0 turn 1 
df1['DT2'] = np.where(df1[['130708_var', '130712_var', '130714_var']].sum(axis=1) > 0, 1, 0)

In [ ]:
def insulin(row):
    if (row['6177-0.0']==3) | (row['6177-0.1']==3) | (row['6177-0.2']==3):
        return 1
    elif pd.isna(row['6177-0.0']) & pd.isna(row['6177-0.1']) & pd.isna(row['6177-0.2']):
        return np.nan
    else:
        return 0

df1['insulin'] = df1.apply(lambda x: insulin(x), axis=1)

In [ ]:
def DMT2_cov(row):
    if (row['insulin']==1) | (row['DT2']==1):
        return 1
    elif pd.isna(row['insulin']) & pd.isna(row['DT2']):
        return np.nan
    else:
        return 0

df1['DMT2_cov'] = df1.apply(lambda x: DMT2_cov(x), axis=1)

## 5. Define Hypertension


In [87]:
# Diabetes tipe II from first occurances
# If date of first ocurrence in I10-I13, I15 > 0 turn 1 
df1['Hypertension'] = np.where(df1[['131286_var', '131288_var', '131290_var', '131292_var', '131294_var']].sum(axis=1) > 0, 1, 0)

In [ ]:
def antihyper(row):
    if (row['6177-0.0']==2) | (row['6177-0.1']==2) | (row['6177-0.2']==2):
        return 1
    elif pd.isna(row['6177-0.0']) & pd.isna(row['6177-0.1']) & pd.isna(row['6177-0.2']):
        return np.nan
    else:
        return 0

df1['antihyper'] = df1.apply(lambda x: antihyper(x), axis=1)

In [ ]:
def HYP_cov(row):
    if (row['antihyper']==1) | (row['Hypertension']==1):
        return 1
    elif pd.isna(row['antihyper']) & pd.isna(row['Hypertension']):
        return np.nan
    else:
        return 0

df1['HYP_cov'] = df1.apply(lambda x: HYP_cov(x), axis=1)

## 6. Define Dyslipidemia 

In [ ]:
def lipid_low_drug(row):
    if (row['6177-0.0']==1) | (row['6177-0.1']==1) | (row['6177-0.2']==1):
        return 1
    elif pd.isna(row['6177-0.0']) & pd.isna(row['6177-0.1']) & pd.isna(row['6177-0.2']):
        return np.nan
    else:
        return 0
    
def HDL_cholesterol(row):
    if row['30760-0.0']<1.0344:
        return 1
    elif row['30760-0.0']>=1.0344:
        return 0
    else:
        return np.nan
    
def Cholesterol(row):
    if row['30690-0.0']>=6.206:
        return 1
    elif row['30690-0.0']<6.206:
        return 0
    else:
        return np.nan
    
def LDL_direct(row):
    if row['30780-0.0']>=4.138:
        return 1
    elif row['30780-0.0']<4.138:
        return 0
    else:
        return np.nan
    
df1['HDL_cholesterol'] = df1.apply(lambda x: HDL_cholesterol(x), axis=1)
df1['Cholesterol'] = df1.apply(lambda x: Cholesterol(x), axis=1)
df1['LDL_direct'] = df1.apply(lambda x: LDL_direct(x), axis=1)
df1['lipid_low_drug'] = df1.apply(lambda x: lipid_low_drug(x), axis=1)

In [ ]:
def DYSL_cov(row):
    if (row['HDL_cholesterol']==1) | (row['Cholesterol']==1) | (row['LDL_direct']==1) | (row['lipid_low_drug']==1):
        return 1
    elif pd.isna(row['HDL_cholesterol']) & pd.isna(row['Cholesterol']) & pd.isna(row['LDL_direct']) & pd.isna(row['lipid_low_drug']):
        return np.nan
    else:
        return 0

df1['DYSL_cov'] = df1.apply(lambda x: DYSL_cov(x), axis=1)

## 7. Define BMI

In [ ]:
def BMI_cov(row):
    if row['21001-0.0']>=30:
        return 1
    elif pd.isna(row['21001-0.0']):
        return np.nan
    else:
        return 0

df1['BMI_cov'] = df1.apply(lambda x: BMI_cov(x), axis=1)
df1['BMI_cov'].isna().value_counts()

## 8. Save output

In [88]:
#Max death age
max(df1[df1['40000-0.0'].notnull()]['40000-0.0'])

'2021-11-12'

In [89]:
### SAVE

df1.to_csv('dataframe_cases_covariables_v2.txt.gz', sep="\t", index=False, compression='gzip')